In [24]:
import jieba
import re
import random
import numpy as np

Data_path = "dataset2"
Topics = ["射雕英雄传", "神雕侠侣", "倚天屠龙记", "天龙八部"]

N = 200 # 段落数，做训练集
D = 500 # 每段的单词数

In [25]:
def data_preprocess_sample(topic):
    """
    输入主题（小说），预处理数据，从中进行文章抽样，然后分词。
    每个小说抽 60 段，前 50 段作为训练集，后 10 段作为测试集。
    """

    data = open("%s/%s.txt" % (Data_path, topic), "r", encoding="utf-8")
    data = data.read()


    ad = ['本书来自www.cr173.com免费txt小说下载站\n更多更新免费电子书请关注www.cr173.com', '----〖新语丝电子文库(www.xys.org)〗', '新语丝电子文库',
          '\u3000', '\n', '。', '？', '！', '，', '；', '：', '、', '《', '》', '“', '”', '‘', '’', '［', '］', '....', '......',
          '『', '』', '（', '）', '…', '「', '」', '\ue41b', '＜', '＞', '+', '\x1a', '\ue42b']
    for a in ad:
        data = data.replace(a, '')

    data_sample = re.findall(r'.{%d}' % D, data)
    data_sample = random.sample(data_sample, N//len(Topics) + 10)
    data_sample = [jieba.lcut(d) for d in data_sample]

    return data_sample[:-10], data_sample[-10:]
    

对四本小说进行预处理，各取出 50+10 段素材，每段长度 500 字。（但是由于编码问题，分词后有不少词语变成方格乱码，实际可用字数不足 500，不过影响不大，也能正常聚类）

In [26]:
training_txt0, testing_txt0 = data_preprocess_sample(Topics[0])
training_txt1, testing_txt1 = data_preprocess_sample(Topics[1])
training_txt2, testing_txt2 = data_preprocess_sample(Topics[2])
training_txt3, testing_txt3 = data_preprocess_sample(Topics[3])

training_txt = training_txt0 + training_txt1 + training_txt2 + training_txt3
testing_txt = testing_txt0 + testing_txt1 + testing_txt2 + testing_txt3

print(len(training_txt))
print(len(testing_txt))


200
40


以下是训练过程。
先给每一个 Document 的每一个词随机分配一个 Topic，然后就可以统计一些量：
- 每个 Document 中，属于每一个 Topic 的词各有多少，记作 $Freq(Topic_j|Document_i)$
- 每个 Document 中，有多少个 Word，记作 $Count(Document_i)$

这两个量可以计算出 Document 中，每个 Topic 的概率分布。即：
$ Freq(Topic_j|Document_i) / Count(Document_i) = P(Topic_j|Document_i)$

- 每个 Topic 中，每个 Word 的频数有多少，记作 $Freq(Word_k|Topic_j)$
- 每个 Topic 中，也多少个 Word，记作 $Count(Topic_j)$

这两个量可以计算出 Topic 中，每个 Word 的概率分布。即：
$ Freq(Word_k|Topic_j) / Count(Topic_j) = P(Word_k|Topic_j)$

------

接下来我们使用上面两个概率分布来反推。假设 Document 中的每一个 Word 是由最大概率来生成的，即：
$ P(Word_k|Document_i) = P(Word_k|Topic_j) \times P(Topic_j|Document_i)$

由此，我们就可以挑出概率最大的 Topic，给每一个词重新分配一个 Topic，然后重复上述步骤，直到无法找到概率更大的新 Topic。

In [27]:

Topic_All = []  # 200 个文章，每个文章中的 500 个词是什么 topic
Topic_count = {}    # 4 个 topic，每个 topic 当中有多少个词
# 4 个 topic，每个 topic 中每个词的词频
Topic0_fre = {}; Topic1_fre = {}; Topic2_fre = {}; Topic3_fre = {}
Doc_fre = []  # 200 个文章，每个文章中，4 个 topic 的词各有多少
Doc_count = []  # 200 个文章，每个文章中有多少个词（500）


i = 0

for data in training_txt:

    topic = []      # 500 个词，每个词初始化一个 topic
    docfre = {}

    for word in data:
        a = random.randint(0, len(Topics)-1)
        topic.append(a)

        if '\u4e00' <= word <= '\u9fa5':
            Topic_count[a] = Topic_count.get(a, 0) + 1  # 统计每个 topic 总词数
            docfre[a] = docfre.get(a, 0) + 1  # 统计每篇文章的词频
            if i < 50:
                Topic0_fre[word] = Topic0_fre.get(word, 0) + 1
            elif i < 100:
                Topic1_fre[word] = Topic1_fre.get(word, 0) + 1
            elif i < 150:
                Topic2_fre[word] = Topic2_fre.get(word, 0) + 1
            else:
                Topic3_fre[word] = Topic3_fre.get(word, 0) + 1
        
        #else:
            #print(word)
    
    Topic_All.append(topic)

    docfre = list(dict(sorted(docfre.items(), key=lambda x: x[0], reverse=False)).values())
    Doc_fre.append(docfre)
    Doc_count.append(sum(docfre))  # [500, 500, ...] 可惜很多乱码无法识别，实际不到 500

    i = i + 1

Topic_count = list(dict(sorted(Topic_count.items(), key=lambda x: x[0], reverse=False)).values())
Doc_fre = np.array(Doc_fre)  # 转为array方便后续计算
Topic_count = np.array(Topic_count)  # 转为array方便后续计算
Doc_count = np.array(Doc_count)  # 转为array方便后续计算
print(Doc_fre.shape)
print(Topic_count)
print(Doc_count.shape)


(200, 4)
[14614 14521 14615 14500]
(200,)


In [28]:
Doc_pro = []  # 每个topic被选中的概率
Doc_pronew = []  # 记录每次迭代后每个topic被选中的新概率

for i in range(len(training_txt)):

    doc = np.divide(Doc_fre[i], Doc_count[i])
    Doc_pro.append(doc)

Doc_pro = np.array(Doc_pro)
print(Doc_pro)

[[0.22775801 0.27402135 0.2633452  0.23487544]
 [0.23928571 0.27857143 0.22857143 0.25357143]
 [0.27666667 0.26       0.26333333 0.2       ]
 [0.23049645 0.30496454 0.24113475 0.22340426]
 [0.2293578  0.30275229 0.22324159 0.24464832]
 [0.22299652 0.24041812 0.28919861 0.24738676]
 [0.28424658 0.23630137 0.24315068 0.23630137]
 [0.21454545 0.22181818 0.25454545 0.30909091]
 [0.26369863 0.24657534 0.22945205 0.26027397]
 [0.27561837 0.30388693 0.19081272 0.22968198]
 [0.25       0.25       0.25684932 0.24315068]
 [0.26388889 0.28472222 0.21875    0.23263889]
 [0.23611111 0.26041667 0.22916667 0.27430556]
 [0.27797834 0.26353791 0.22021661 0.23826715]
 [0.20477816 0.23208191 0.25938567 0.30375427]
 [0.33670034 0.23905724 0.22222222 0.2020202 ]
 [0.24342105 0.23684211 0.27631579 0.24342105]
 [0.24914676 0.2559727  0.25938567 0.23549488]
 [0.26885246 0.21967213 0.29508197 0.21639344]
 [0.24422442 0.30363036 0.23432343 0.21782178]
 [0.27891156 0.21088435 0.2414966  0.26870748]
 [0.23920266 

In [29]:
stop = 0  # 迭代停止标志
loopcount = 1  # 迭代次数
while stop == 0:
    i = 0   # 文章数目
    for data in training_txt:
        top = Topic_All[i]
        for w in range(len(data)):
            word = data[w]
            pro = []
            topfre = []
            if '\u4e00' <= word <= '\u9fa5':
                #for j in range(len(Topics)):
                #    exec('topfre.append(Topic_fre{}.get(word, 0))'.format(j))  # 读取该词语在每个topic中出现的频数
                topfre.append(Topic0_fre.get(word, 0))
                topfre.append(Topic1_fre.get(word, 0))
                topfre.append(Topic2_fre.get(word, 0))
                topfre.append(Topic3_fre.get(word, 0))
                
                pro = Doc_pro[i] * topfre / Topic_count  # 计算每篇文章选中各个topic的概率乘以该词语在每个topic中出现的概率，得到该词出现的概率向量
                m = np.argmax(pro)  # 认为该词是由上述概率之积最大的那个topic产生的
                Doc_fre[i][top[w]] -= 1  # 更新每个文档有多少各个topic的词
                Doc_fre[i][m] += 1
                Topic_count[top[w]] -= 1  # 更新每个topic的总词数
                Topic_count[m] += 1
                exec('Topic{}_fre[word] = Topic{}_fre.get(word, 0) - 1'.format(top[w], top[w]))  # 更新每个topic该词的频数
                exec('Topic{}_fre[word] = Topic{}_fre.get(word, 0) + 1'.format(m, m))
                top[w] = m
        Topic_All[i] = top
        i += 1
    print(Doc_fre, 'new')
    print(Topic_count, 'new')
    if loopcount == 1:  # 计算新的每篇文章选中各个topic的概率
        for i in range(len(training_txt)):
            doc = np.divide(Doc_fre[i], Doc_count[i])
            Doc_pronew.append(doc)
        Doc_pronew = np.array(Doc_pronew)
    else:
        for i in range(len(training_txt)):
            doc = np.divide(Doc_fre[i], Doc_count[i])
            Doc_pronew[i] = doc
    print(Doc_pro)
    print(Doc_pronew)
    if (Doc_pronew == Doc_pro).all():  # 如果每篇文章选中各个topic的概率不再变化，则认为模型已经训练完毕
        stop = 1
    else:
        Doc_pro = Doc_pronew.copy()
    loopcount += 1
print(Doc_pronew)  # 输出最终训练的到的每篇文章选中各个topic的概率
print(loopcount)  # 输出迭代次数
print(Topic_count)
print('模型训练完毕！')

[[154  55  46  26]
 [139  67  15  59]
 [194  41  50  15]
 [131  87  40  24]
 [137 130  14  46]
 [131  39  66  51]
 [201  25  27  39]
 [133  13  25 104]
 [148  63  24  57]
 [177  61   8  37]
 [159  47  35  51]
 [148  71  17  52]
 [145  60  31  52]
 [158  64  18  37]
 [115  54  30  94]
 [204  48  25  20]
 [119  70  58  57]
 [145  62  40  46]
 [158  32  63  52]
 [119  95  46  43]
 [159  33  34  68]
 [149  61  29  62]
 [125  59  48  43]
 [138  42  74  55]
 [152  47  35  57]
 [134  68  57  48]
 [128  65  45  62]
 [119  84  38  62]
 [150  31  32  66]
 [147  58  37  61]
 [170  32  42  51]
 [134  61  45  38]
 [167  52  24  59]
 [129  82  33  62]
 [132  57  40  51]
 [134  42  38  68]
 [122  66  36  52]
 [137  65  42  29]
 [132 105  31  51]
 [141  75  35  58]
 [147  60  27  66]
 [145  46  44  52]
 [145  36  51  48]
 [140  56  40  51]
 [135  66  42  50]
 [148  58  30  57]
 [142  46  56  49]
 [140  62  47  52]
 [133  54  59  43]
 [122  76  43  54]
 [ 26 166  36  58]
 [ 20 190  44  39]
 [ 27 176  4

In [30]:
print(Doc_pronew.shape)
print(np.argmax(Doc_pronew, 1)[:50]) 
print(np.argmax(Doc_pronew, 1)[50:100]) 
print(np.argmax(Doc_pronew, 1)[100:150]) 
print(np.argmax(Doc_pronew, 1)[150:200]) 
# 训练结果非常神奇，能够完美分类

(200, 4)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3]


模型实际训练出了不同 Topic 中每一个 Word 的词频以及总词数，也就是 Topic 中，Word 的概率分布，在测试中直接调用就行。

In [31]:
Doc_count_test = []  # 每篇文章中有多少个词
Doc_fre_test = []  # 每篇文章有多少各个topic的词
Topic_All_test = []  # 每篇文章中的每个词来自哪个topic
i = 0
for data in testing_txt:
    topic = []
    docfre = {}
    for word in data:
        a = random.randint(0, len(Topics) - 1)  # 为每个单词赋予一个随机初始topic
        topic.append(a)
        if '\u4e00' <= word <= '\u9fa5':
            docfre[a] = docfre.get(a, 0) + 1  # 统计每篇文章的词频
    Topic_All_test.append(topic)
    docfre = list(dict(sorted(docfre.items(), key=lambda x: x[0], reverse=False)).values())
    Doc_fre_test.append(docfre)
    Doc_count_test.append(sum(docfre))  # 统计每篇文章的总词数
    i += 1
# print(Topic_All[0])
Doc_fre_test = np.array(Doc_fre_test)
Doc_count_test = np.array(Doc_count_test)
print(Doc_fre_test)
print(Doc_count_test)


Doc_pro_test = []  # 每个topic被选中的概率
Doc_pronew_test = []  # 记录每次迭代后每个topic被选中的新概率
for i in range(len(testing_txt)):
    doc = np.divide(Doc_fre_test[i], Doc_count_test[i])
    Doc_pro_test.append(doc)
Doc_pro_test = np.array(Doc_pro_test)
print(Doc_pro_test)


stop = 0  # 迭代停止标志
loopcount = 1  # 迭代次数
while stop == 0:
    i = 0
    for data in testing_txt:
        top = Topic_All_test[i]
        for w in range(len(data)):
            word = data[w]
            pro = []
            topfre = []
            if '\u4e00' <= word <= '\u9fa5':
                # for j in range(len(data_txt)):
                # exec('topfre.append(Topic_fre{}.get(word, 0))'.format(j))  # 读取该词语在每个topic中出现的频数
                topfre.append(Topic0_fre.get(word, 0))
                topfre.append(Topic1_fre.get(word, 0))
                topfre.append(Topic2_fre.get(word, 0))
                topfre.append(Topic3_fre.get(word, 0))
                
                pro = Doc_pro_test[i] * topfre / Topic_count  # 计算每篇文章选中各个topic的概率乘以该词语在每个topic中出现的概率，得到该词出现的概率向量
                m = np.argmax(pro)  # 认为该词是由上述概率之积最大的那个topic产生的
                Doc_fre_test[i][top[w]] -= 1  # 更新每个文档有多少各个topic的词
                Doc_fre_test[i][m] += 1
                top[w] = m
        Topic_All_test[i] = top
        i += 1
    print(Doc_fre_test, 'new')
    if loopcount == 1:  # 计算新的每篇文章选中各个topic的概率
        for i in range(len(testing_txt)):
            doc = np.divide(Doc_fre_test[i], Doc_count_test[i])
            Doc_pronew_test.append(doc)
        Doc_pronew_test = np.array(Doc_pronew_test)
    else:
        for i in range(len(testing_txt)):
            doc = np.divide(Doc_fre_test[i], Doc_count_test[i])
            Doc_pronew_test[i] = doc
    print(Doc_pro_test)
    print(Doc_pronew_test)
    if (Doc_pronew_test == Doc_pro_test).all():  # 如果每篇文章选中各个topic的概率不再变化，则认为训练集已分类完毕
        stop = 1
    else:
        Doc_pro_test = Doc_pronew_test.copy()
    loopcount += 1
print(Doc_pronew)
print(Doc_pronew_test)
print(loopcount)
print('测试集测试完毕！')

[[82 85 74 63]
 [81 66 71 75]
 [58 61 76 90]
 [73 72 72 76]
 [70 71 77 78]
 [74 69 74 78]
 [73 70 73 66]
 [65 76 71 77]
 [85 76 74 76]
 [76 84 80 62]
 [73 72 81 67]
 [73 70 67 69]
 [73 74 76 70]
 [75 69 60 77]
 [76 66 72 74]
 [71 81 69 60]
 [27 34 41 33]
 [68 80 72 76]
 [56 50 49 47]
 [54 57 69 77]
 [78 64 68 76]
 [71 77 79 79]
 [73 63 71 68]
 [98 72 69 64]
 [73 66 73 72]
 [71 65 70 70]
 [70 76 70 78]
 [71 80 74 66]
 [60 71 78 74]
 [81 80 65 84]
 [74 59 63 82]
 [79 65 68 76]
 [87 95 74 74]
 [98 72 67 71]
 [67 82 75 83]
 [75 77 74 85]
 [81 78 76 59]
 [86 84 78 74]
 [77 72 63 69]
 [63 77 74 79]]
[304 293 285 293 296 295 282 289 311 302 293 279 293 281 288 281 135 296
 202 257 286 306 275 303 284 276 294 291 283 310 278 288 330 308 307 311
 294 322 281 293]
[[0.26973684 0.27960526 0.24342105 0.20723684]
 [0.27645051 0.22525597 0.24232082 0.2559727 ]
 [0.20350877 0.21403509 0.26666667 0.31578947]
 [0.24914676 0.24573379 0.24573379 0.25938567]
 [0.23648649 0.23986486 0.26013514 0.26351351]


In [33]:
result = []
for k in range(len(testing_txt)):
    pro = []
    for i in range(len(Topics)):
        dis = 0
        for j in range(len(Topics)):
            dis += (Doc_pro[i][j] - Doc_pro_test[k][j])**2  # 计算欧式距离
        pro.append(dis)
    m = pro.index(min(pro))
    print(pro)
    result.append(m)
#print(files)
print(result[:10])
print(result[10:20])
print(result[20:30])
print(result[30:])
# 测试分类成功率相当感人
# 可以考虑用测试得到的概率分布去训练一个 full-connect 模型试试，能不能很好地拟合出一个分类模型

[0.07192750458552401, 0.02626937998190965, 0.029875988765774096, 0.039422613154571635]
[0.051836742075738795, 0.015360861714719311, 0.01558090575053616, 0.023705289210965298]
[0.04265709021493748, 0.008933400229898041, 0.011684641428131737, 0.02050285347401526]
[0.106995744460502, 0.04680718857101017, 0.047103618109833705, 0.0642851990625776]
[0.0394754111767634, 0.0071037812495341425, 0.009466528690853026, 0.0185719461952746]
[0.06408542377625531, 0.022640492996968947, 0.022973966612403852, 0.0316438393040718]
[0.03367742982028386, 0.010873751115052928, 0.010793249836527349, 0.011593984206025858]
[0.038922855323155786, 0.008874890379518375, 0.009420294031175133, 0.01589410326532245]
[0.0757391320931328, 0.025652755115755824, 0.027837929485610972, 0.04234179394214987]
[0.0655675435159287, 0.023351550325844985, 0.023788739285314005, 0.032737727287103446]
[0.10587169851191122, 0.047539454359945235, 0.05356154617733206, 0.0662317142576145]
[0.08247399908901615, 0.051092047197728266, 0.047

In [37]:
print(Doc_pronew_test.shape)

(40, 4)


In [45]:
label = [0 for i in range(10)] + [1 for i in range(10)] + [2 for i in range(10)] + [3 for i in range(10)]
train_index = random.sample(range(40), 30)
test_index = [i for i in range(40) if i not in train_index]

print(train_index)
print(test_index)

labels = np.zeros(Doc_pronew_test.shape)
for i in range(len(label)):
    labels[i, label[i]] = 1

print(labels)


[17, 1, 4, 7, 27, 30, 12, 3, 36, 11, 22, 2, 19, 13, 25, 18, 14, 26, 39, 23, 34, 29, 10, 33, 0, 35, 9, 38, 28, 15]
[5, 6, 8, 16, 20, 21, 24, 31, 32, 37]
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [65]:

train_data = Doc_pronew_test[train_index, :]
train_label = labels[train_index]
test_data = Doc_pronew_test[test_index, :]
test_label = labels[test_index]

import torch
import torch.nn as nn
from torch.autograd import Variable

train_data = torch.from_numpy(train_data.astype(np.float32))
train_label = torch.from_numpy(train_label.astype(np.float32))
test_data = torch.from_numpy(test_data.astype(np.float32))
test_label = torch.from_numpy(test_label.astype(np.float32))

print(train_label.shape)
print(test_label.shape)


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 4)

    def forward(self, x):

        x = self.fc1(x)
        x = self.fc2(x)
        x = nn.Softmax(dim=1)(x)

        return x


torch.Size([30, 4])
torch.Size([10, 4])


In [66]:
model = Classifier()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1,
                             weight_decay=1e-5)
criterion = nn.MSELoss()

for i in range(200):
    total_loss = 0

    optimizer.zero_grad()

    output = model(train_data)
    loss = criterion(output, train_label)

    loss.backward()
    optimizer.step()

    total_loss += loss.data

    if i % 10 == 0:
        print('epoch [{}/{}], loss:{:.4f}'
            .format(i, 10, total_loss))


torch.save(model.state_dict(), 'classifier.pth')


epoch [0/10], loss:0.1901
epoch [10/10], loss:0.1653
epoch [20/10], loss:0.1213
epoch [30/10], loss:0.0952
epoch [40/10], loss:0.0790
epoch [50/10], loss:0.0719
epoch [60/10], loss:0.0702
epoch [70/10], loss:0.0694
epoch [80/10], loss:0.0686
epoch [90/10], loss:0.0680
epoch [100/10], loss:0.0672
epoch [110/10], loss:0.0688
epoch [120/10], loss:0.0678
epoch [130/10], loss:0.0668
epoch [140/10], loss:0.0655
epoch [150/10], loss:0.0653
epoch [160/10], loss:0.0649
epoch [170/10], loss:0.0647
epoch [180/10], loss:0.0644
epoch [190/10], loss:0.0642


In [74]:

output_test = model(test_data)
print(output_test)
print(test_label)

print(np.argmax(output_test.detach().numpy(), 1))
print(np.argmax(test_label.numpy()), 1)

loss_test = criterion(output_test, test_label)

print(loss_test)

# 正确率还算凑或，比直接的用距离来衡量好多了

tensor([[6.1482e-01, 1.0057e-01, 8.6229e-03, 2.7599e-01],
        [2.2202e-02, 9.7611e-01, 5.1795e-04, 1.1664e-03],
        [5.1315e-01, 9.1409e-04, 2.0330e-04, 4.8574e-01],
        [1.5308e-03, 9.9837e-01, 2.0830e-05, 7.9241e-05],
        [2.9148e-04, 3.8857e-04, 9.9817e-01, 1.1547e-03],
        [2.0740e-02, 8.6342e-02, 7.4074e-01, 1.5217e-01],
        [5.8022e-05, 7.9794e-02, 9.2004e-01, 1.0483e-04],
        [5.7862e-01, 1.0623e-02, 2.1595e-03, 4.0860e-01],
        [9.7870e-03, 9.2726e-01, 6.7611e-07, 6.2956e-02],
        [4.8867e-02, 5.5900e-04, 7.8613e-04, 9.4979e-01]],
       grad_fn=<SoftmaxBackward>)
tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]])
[0 1 0 1 2 2 2 0 1 3]
0 1
tensor(0.1289, grad_fn=<MseLossBackward>)
